In [1]:
from pathlib import Path
import SimpleITK as sitk
import numpy as np
%matplotlib widget
import matplotlib.pyplot as plt

import multiscale.itk.metadata as meta
import multiscale.plotting as myplot
from multiscale.itk.itk_plotting import MultiImageDisplay, RegistrationPointDataAcquisition
import multiscale.itk.itk_plotting as iplot

In [2]:
us_path = Path(r'F:\Research\LINK\Phantom Trials\2018-12-07\Interleaved_5-1V_Sens0-6', 'Reslice of fused_tp_0_ch_0.tif')
mpm_path = Path(r'F:\Research\LINK\Phantom Trials\2018-12-04', 'MPM Grid shrunk 16x in xy.tif')

In [3]:
us_image = sitk.ReadImage(str(us_path))
us_image.SetSpacing([25, 25, 25])
mpm_image = sitk.ReadImage(str(mpm_path))
mpm_image.SetSpacing([16, 16, 7.5])

In [4]:
us_window = myplot.auto_window_level(sitk.GetArrayFromImage(us_image), return_image=False)
mpm_window = myplot.auto_window_level(sitk.GetArrayFromImage(mpm_image), bins=200, upper_limit_fraction=0.3, lower_limit_fraction=0.05, return_image=False)

In [5]:
MultiImageDisplay([us_image, mpm_image], title_list=['US', 'MPM'])

Box(children=(IntSlider(value=20, description='image slice:', max=40), IntSlider(value=0, description='image s…

FigureCanvasNbAgg()

In [6]:
points = RegistrationPointDataAcquisition(us_image, mpm_image)

FigureCanvasNbAgg()

In [8]:
fixed_image_points, moving_image_points = points.get_points_flat()
print(fixed_image_points)

[19448.77564964158, 8804.428539426528, 500.0, 19863.14840949821, 5655.195564516129, 500.0, 8757.9584453405, 8224.306675627242, 500.0, 12984.560595878136, 24136.22065412187, 500.0]


In [9]:

initial_transform = sitk.LandmarkBasedTransformInitializer(sitk.VersorRigid3DTransform(), 
                                                                fixed_image_points, 
                                                                moving_image_points)
print(initial_transform)

itk::simple::Transform
 VersorRigid3DTransform (00000293B31A8B70)
   RTTI typeinfo:   class itk::VersorRigid3DTransform<double>
   Reference Count: 1
   Modified Time: 1160
   Debug: Off
   Object Name: 
   Observers: 
     none
   Matrix: 
     0.99915 -0.0412237 0 
     0.0412237 0.99915 0 
     0 0 1 
   Offset: [-5963.65, -3603.06, -500]
   Center: [15263.6, 11705, 500]
   Translation: [-6459.14, -2983.79, -500]
   Inverse: 
     0.99915 0.0412237 0 
     -0.0412237 0.99915 0 
     0 0 1 
   Singular: 0
   Versor: [ 0, 0, 0.0206162, 0.999787 ]



In [10]:
registered_image = sitk.Resample(mpm_image, us_image, initial_transform, sitk.sitkLinear, 0.0, mpm_image.GetPixelID())

In [11]:
MultiImageDisplay([us_image, registered_image], title_list=['US', 'MPM'])

Box(children=(IntSlider(value=20, description='image slice:', max=40), IntSlider(value=20, description='image …

FigureCanvasNbAgg()

In [13]:
overlay = iplot.overlay_images(us_image, registered_image, slice=20)
fig, ax = plt.subplots()
ax.imshow(overlay)

FigureCanvasNbAgg()